In [1]:
import csv
import os
import math
import pandas as pd

cur_wd = os.getcwd()
print(cur_wd)

D:\janko_machine_learning\playground


In [2]:
crsp = pd.read_csv(os.path.join(r'D:\\','etf_universe.csv'))
print(crsp.head(5))

C:\Users\jjanko\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5,9,18,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   PERMNO      date  NAMEENDT  SHRCD  EXCHCD SICCD    NCUSIP TICKER  \
0   10001  19970131       NaN   11.0     3.0  4920  29274A10   EWST   
1   10001  19970228       NaN   11.0     3.0  4920  29274A10   EWST   
2   10001  19970331       NaN   11.0     3.0  4920  29274A10   EWST   
3   10001  19970430       NaN   11.0     3.0  4920  29274A10   EWST   
4   10001  19970530       NaN   11.0     3.0  4920  29274A10   EWST   

            COMNAM SHRCLS  ... CFACSHR  ALTPRC SPREAD    ALTPRCDT      RETX  \
0  ENERGY WEST INC    NaN  ...     1.5   8.625    NaN  19970131.0  0.061538   
1  ENERGY WEST INC    NaN  ...     1.5   8.625    NaN  19970228.0  0.000000   
2  ENERGY WEST INC    NaN  ...     1.5   8.625    NaN  19970331.0  0.000000   
3  ENERGY WEST INC    NaN  ...     1.5   8.625    NaN  19970430.0  0.000000   
4  ENERGY WEST INC    NaN  ...     1.5   8.625    NaN  19970530.0  0.000000   

     vwretd    vwretx    ewretd    ewretx    sprtrn  
0  0.053441  0.052358  0.063831  0.063007  0

In [3]:
crsp = crsp.loc[crsp['SHRCD'] == 73., :]
crsp = crsp.loc[crsp['EXCHCD'].isin([1., 2., 3., 4.]), :]
crsp = crsp.loc[crsp['PRC'] > 3.0]
crsp['market_cap'] = (crsp['PRC'] * crsp['SHROUT']).abs()
crsp['RET'] = crsp['RET'].astype(str)
crsp = crsp[~crsp['RET'].isin(['C', 'B', '-66.0', '-99.0', '-77.0', '-88.0'])]
crsp['RET'] = crsp['RET'].astype('float')

print(crsp)

         PERMNO      date    NAMEENDT  SHRCD  EXCHCD SICCD    NCUSIP TICKER  \
5003      10113  20100831         NaN   73.0     4.0  6726  00768Y20   AADR   
5004      10113  20100930         NaN   73.0     4.0  6726  00768Y20   AADR   
5005      10113  20101029         NaN   73.0     4.0  6726  00768Y20   AADR   
5006      10113  20101130         NaN   73.0     4.0  6726  00768Y20   AADR   
5007      10113  20101231         NaN   73.0     4.0  6726  00768Y20   AADR   
5008      10113  20110131         NaN   73.0     4.0  6726  00768Y20   AADR   
5009      10113  20110228         NaN   73.0     4.0  6726  00768Y20   AADR   
5011      10113  20110429         NaN   73.0     4.0  6726  00768Y20   AADR   
5012      10113  20110531         NaN   73.0     4.0  6726  00768Y20   AADR   
5013      10113  20110630         NaN   73.0     4.0  6726  00768Y20   AADR   
5014      10113  20110729         NaN   73.0     4.0  6726  00768Y20   AADR   
5015      10113  20110831         NaN   73.0     4.0

In [14]:
sectors = crsp[crsp['TICKER'].isin(['XLY', 'XLF', 'XLE', 'XLI', 'XLP', 'XME', 'XLU','XLV', 'XLK','XLB','XLC', 'SPY'])]
factor = crsp[crsp['TICKER'].isin(['SPY', 'IWM', 'IVV', 'AGG', 'IJR', 'IWF', 'IWD', 'IWV', 'IWR', 'IWS', 'IWP', 'IWN', 'IWO'])]
"""
spy- s&p 500
IWWM-  iShares Russell 2000 ETF 
IVV- iShares Core S&P 500 ETF large cap
AGG- bond
IJR- iShares Core S&P Small-Cap ETF 
IWF- iShares Russell 1000 Growth ETF 
IWD iShares Russell 1000 Value ETF 
IWV- iShares Russell 3000 ETF 
"""

'\nspy- s&p 500\nIWWM-  iShares Russell 2000 ETF \nIVV- iShares Core S&P 500 ETF large cap\nAGG- bond\nIJR- iShares Core S&P Small-Cap ETF \nIWF- iShares Russell 1000 Growth ETF \nIWD iShares Russell 1000 Value ETF \nIWV- iShares Russell 3000 ETF \n'

In [17]:
print("returns sector")
sector_pivot = sectors.pivot_table(values = 'RET', index = 'date', columns = ['TICKER'])
sector_pivot = sector_pivot.loc[sector_pivot.index > 20010000]
print(sector_pivot.head(5))
print("returns factors")
factors_pivot = factor.pivot_table(values = 'RET', index = 'date', columns = ['TICKER']).dropna(how = 'all')
factors_pivot = factors_pivot.loc[factors_pivot.index > 20030000]
factors_pivot.loc[:, 'MKT'] = factors_pivot.loc[:, 'SPY'] 
factors_pivot.loc[:, 'SMB'] = factors_pivot.loc[:, 'IJR'] -  factors_pivot.loc[:, 'IVV']
factors_pivot.loc[:, 'HML'] = factors_pivot.loc[:, 'IWD'] -  factors_pivot.loc[:, 'IWF']
factors_pivot.loc[:,'rms5'] = factors_pivot.loc[:, 'IWR'] -  factors_pivot.loc[:, 'SPY']
#Russell Midcap minus S&P 500 
factors_pivot.loc[:,'r2rm'] = factors_pivot.loc[:, 'IWM'] -  factors_pivot.loc[:, 'IWR']
#Russell 2000 minus Russell Midcap 
factors_pivot.loc[:,'s5vs5g'] = factors_pivot.loc[:, 'IWM'] -  factors_pivot.loc[:, 'IWR']
#S&P 500 Value minus S&P 500 Growth
factors_pivot.loc[:,'rmvrmg'] = factors_pivot.loc[:, 'IWS'] -  factors_pivot.loc[:, 'IWP']
#Russell Midcap Value minus Russell Midcap Growth 
factors_pivot.loc[:,'r2vr2g'] = factors_pivot.loc[:, 'IWN'] -  factors_pivot.loc[:, 'IWO']
#Russell 2000 Value minus Russell 2000 Growth
factors_pivot_five = factors_pivot[['MKT','r2rm','s5vs5g','rmvrmg','r2vr2g' ]]
factors_pivot = factors_pivot[['MKT', 'SMB', 'HML']]

print(factors_pivot.head(5))

print("returns")
crsp_pivot = crsp.pivot_table(values='RET',index='date',columns=['PERMNO'])
print(crsp_pivot.head(5))

print("market cap")
crsp_mkt = crsp.pivot_table(values='market_cap',index='date',columns=['PERMNO'])
print(crsp_mkt.head(5))

returns sector
TICKER         SPY       XLB  XLC       XLE       XLF       XLI       XLK  \
date                                                                        
20010131  0.044459 -0.026696  NaN -0.031864 -0.002373  0.003840  0.191218   
20010228 -0.095388  0.012950  NaN -0.012138 -0.067278 -0.059930 -0.249062   
20010330 -0.056026 -0.049978  NaN -0.029723 -0.029976 -0.095653 -0.114602   
20010430  0.070443  0.106660  NaN  0.103257  0.036172  0.114329  0.161694   
20010531  0.008326  0.038462  NaN -0.009448  0.044000  0.041850 -0.040958   

TICKER         XLP       XLU       XLV       XLY  XME  
date                                                   
20010131 -0.072910 -0.056271  0.072294  0.116288  NaN  
20010228  0.006042  0.020754 -0.019165 -0.071404  NaN  
20010330 -0.069352 -0.014553 -0.064023 -0.014582  NaN  
20010430  0.019418  0.048258  0.073825  0.046378  NaN  
20010531  0.014682  0.007012  0.038194  0.023810  NaN  
returns factors
TICKER         MKT       SMB       HM

In [18]:
crsp_pivot = crsp_pivot.dropna(axis = 1, how = 'all')
crsp_pivot = crsp_pivot.dropna(axis = 0, how = 'all')
crsp_mkt = crsp_mkt.dropna(axis = 1, how = 'all')
crsp_mkt = crsp_mkt.dropna(axis = 0, how = 'all')
print(crsp_pivot)

PERMNO       10113  10151  11061     11182  11196     11264     11363  \
date                                                                    
19970131       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19970228       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19970331       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19970430       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19970530       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19970630       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19970731       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19970829       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19970930       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19971031       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19971128       NaN    NaN    NaN       NaN    NaN       NaN       NaN   
19971231       NaN    NaN    NaN       NaN    NaN  

In [19]:
crsp_pivot = crsp_pivot[crsp_pivot.count(axis = 1) > 250]
crsp_mkt = crsp_mkt.loc[crsp_pivot.index,:]

In [20]:
ranking = crsp_mkt.rank(axis = 1, ascending = False).shift(1).dropna(axis = 0, how = 'all')
ranking = ranking[ranking <= 150].dropna(axis = 1, how = 'all')

In [21]:
crsp_pivot = crsp_pivot.loc[ranking.index, ranking.columns][ranking.notnull()]

In [22]:
crsp_pivot.to_csv('ETF.csv')
sector_pivot.to_csv('Sector_ETF.csv')
factors_pivot.to_csv('Three_Factor_ETF.csv')
factors_pivot_five.to_csv('Five_Factor_ETF.csv')

In [23]:
# Replicate the Petajisto Cremers 4 factor model